In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
!pip install torchvision

In [2]:
import argparse
import matplotlib.pyplot as plt
import cv2
import os
from objRemove import ObjectRemove
from models.deepFill import Generator
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights
from google.colab.patches import cv2_imshow

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\남현석\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [3]:
def lengthFinger(land, wid, hei):
    return ((land[2].x*wid - land[1].x*wid)**2 + (land[2].y*hei - land[1].y*hei)**2)**0.5

In [ ]:
def myObjRemove(image_file, minx, miny, maxx, maxy):
  ######################################################
  #creating Mask-RCNN model and load pretrained weights#
  ######################################################
  for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/졸업연구/src/models'):
      if f.endswith('.pth'):
          deepfill_weights_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/졸업연구/src/models', f)
  print("Creating rcnn model")
  weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
  transforms = weights.transforms()
  rcnn = maskrcnn_resnet50_fpn(weights=weights, progress=False)
  rcnn = rcnn.eval()

  #########################
  #create inaptining model#
  #########################
  print('Creating deepfil model')
  deepfill = Generator(checkpoint=deepfill_weights_path, return_flow=True)
  ######################
  #create ObjectRemoval#
  ######################
  model = ObjectRemove(segmentModel=rcnn,
                          rcnn_transforms=transforms,
                          inpaintModel=deepfill,
                          image_file=image_file,
                          minx=minx,
                          miny=miny,
                          maxx=maxx,
                          maxy=maxy)

  # model.minx = minx
  # model.miny = miny
  # model.maxx = maxx
  # model.maxy = maxy


  #####
  #run#
  #####
  output = model.run()

  #################
  #display results#
  #################
  img = cv2.cvtColor(model.image_orig[0].permute(1,2,0).numpy(),cv2.COLOR_RGB2BGR)
  boxed = cv2.rectangle(img, (model.box[0], model.box[1]),(model.box[2], model.box[3]), (0,255,0),2)
  boxed = cv2.cvtColor(boxed,cv2.COLOR_BGR2RGB)

  # fig,axs = plt.subplots(1,3,layout='constrained')
  # axs[0].imshow(boxed)
  # axs[0].set_title('Original Image Bounding Box')
  # axs[1].imshow(model.image_masked.permute(1,2,0).detach().numpy())
  # axs[1].set_title('Masked Image')
  # axs[2].imshow(output)
  # axs[2].set_title('Inpainted Image')
  # plt.show()
  # cv2_imshow(output)
  return output


In [4]:
# 이미지 파일의 경우을 사용하세요.:
# IMAGE_FILES = ["/content/drive/MyDrive/Colab Notebooks/졸업연구/img/test/test" + str(i) + ".jpg" for i in range(7)]
directory_path = '/content/drive/MyDrive/Colab Notebooks/졸업연구/img/test/'
IMAGE_FILES = os.listdir(directory_path)
ranges = []

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:

    for idx, file in enumerate(IMAGE_FILES):
        # 이미지를 읽어 들이고, 보기 편하게 이미지를 좌우 반전합니다.
        # print(file)
        image = cv2.flip(cv2.imread(directory_path + file), 1)
        # 작업 전에 BGR 이미지를 RGB로 변환합니다.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # 손으로 프린트하고 이미지에 손 랜드마크를 그립니다.
        if not results.multi_hand_landmarks:
            continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()

        range_dict = {"minx":[], "miny":[], "maxx":[], "maxy":[]}
        for hand_landmarks in results.multi_hand_landmarks:

            len_fing = lengthFinger(hand_landmarks.landmark, image_width, image_height)
            minx, miny, maxx, maxy = image_width, image_height, 0, 0

            for land in hand_landmarks.landmark:
                if minx > land.x*image_width: minx = land.x*image_width
                if maxx < land.x*image_width: maxx = land.x*image_width
                if miny > land.y*image_height: miny = land.y*image_height
                if maxy < land.y*image_height: maxy = land.y*image_height
                
            minx -= len_fing/2
            miny -= len_fing/2
            maxx += len_fing/2
            maxy += len_fing/2
            minx, miny, maxx, maxy = int(minx), int(miny), int(maxx), int(maxy)
            # cv2.rectangle(annotated_image, (minx, miny), (maxx, maxy), (0, 0, 255), 3)
            
            range_dict['minx'].append(minx)
            range_dict['miny'].append(miny)
            range_dict['maxx'].append(maxx)
            range_dict['maxy'].append(maxy)
        
        for i in range(len(range_dict['minx'])):
            annotated_image = myObjRemove(annotated_image,
                                          range_dict['minx'][i],
                                          range_dict['miny'][i],
                                          range_dict['maxx'][i],
                                          range_dict['maxy'][i])
        
        ranges.append(range_dict)

        # cv2.imwrite('../img/testbox/testbox' + str(idx) + '.png', cv2.flip(annotated_image, 1))
        # plt.imshow(cv2.flip(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB), 1))
        print("\n\n"+file+" is done")
        cv2_imshow(cv2.flip(annotated_image, 1))

print(ranges)


../img/hand0.png
../img/hand1.png
../img/hand2.png
[{'minx': [-7, 350], 'miny': [-24, 58], 'maxx': [388, 801], 'maxy': [534, 561]}, {'minx': [147], 'miny': [237], 'maxx': [368], 'maxy': [582]}, {'minx': [22], 'miny': [67], 'maxx': [250], 'maxy': [362]}]


In [5]:
# 이미지 파일의 경우을 사용하세요.:
IMAGE_FILES = ["../img/test/test" + str(i) + ".jpg" for i in range(7)]
ranges = []

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    
    for idx, file in enumerate(IMAGE_FILES):
        # 이미지를 읽어 들이고, 보기 편하게 이미지를 좌우 반전합니다.
        print(file)
        image = cv2.flip(cv2.imread(file), 1)
        # 작업 전에 BGR 이미지를 RGB로 변환합니다.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # 손으로 프린트하고 이미지에 손 랜드마크를 그립니다.
        if not results.multi_hand_landmarks:
            continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        
        range_dict = {"minx":[], "miny":[], "maxx":[], "maxy":[]}
        for hand_landmarks in results.multi_hand_landmarks:
            
            len_fing = lengthFinger(hand_landmarks.landmark, image_width, image_height)
            minx, miny, maxx, maxy = image_width, image_height, 0, 0
            
            for land in hand_landmarks.landmark:
                if minx > land.x*image_width: minx = land.x*image_width
                if maxx < land.x*image_width: maxx = land.x*image_width
                if miny > land.y*image_height: miny = land.y*image_height
                if maxy < land.y*image_height: maxy = land.y*image_height
                
            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            minx -= len_fing/2
            miny -= len_fing/2
            maxx += len_fing/2
            maxy += len_fing/2
            minx, miny, maxx, maxy = int(minx), int(miny), int(maxx), int(maxy)
            cv2.rectangle(annotated_image, (minx, miny), (maxx, maxy), (0, 0, 255), 1)
            
            range_dict['minx'].append(minx)
            range_dict['miny'].append(miny)
            range_dict['maxx'].append(maxx)
            range_dict['maxy'].append(maxy)
        ranges.append(range_dict)
            
        cv2.imwrite('../img/testbox/testbox' + str(idx) + '.png', cv2.flip(annotated_image, 1))

print(ranges)


../img/test/test0.jpg
../img/test/test1.jpg
../img/test/test2.jpg
../img/test/test3.jpg
../img/test/test4.jpg
../img/test/test5.jpg
../img/test/test6.jpg
[{'minx': [412], 'miny': [327], 'maxx': [626], 'maxy': [615]}, {'minx': [353, 520], 'miny': [508, 431], 'maxx': [447, 665], 'maxy': [600, 549]}, {'minx': [425], 'miny': [761], 'maxx': [513], 'maxy': [823]}, {'minx': [908, 127], 'miny': [353, 315], 'maxx': [1002, 221], 'maxy': [465, 436]}, {'minx': [689, 586], 'miny': [885, 708], 'maxx': [877, 711], 'maxy': [1006, 843]}, {'minx': [681, 203], 'miny': [979, 920], 'maxx': [764, 302], 'maxy': [1089, 1050]}, {'minx': [105, 695], 'miny': [896, 729], 'maxx': [279, 850], 'maxy': [1106, 900]}]
